<a href="https://colab.research.google.com/github/Okelezo/LSTM-Bayes-Optimizer/blob/main/LSTM_Bayes_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from bayes_opt import BayesianOptimization

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load data
price = pd.read_csv('/content/AMZN.csv')
price = price['Close'].values.astype(float)

# Test, validation, and training splits
test_data_size = 100
valid_data_size = 100

train_data = price[:-2*test_data_size]
valid_data = price[-2*test_data_size:-test_data_size]
test_data = price[-test_data_size:]

# Normalize data
scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_data.reshape(-1, 1))

# Convert to tensors
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1).to(device)
valid_data_normalized = scaler.transform(valid_data.reshape(-1, 1))
test_data_normalized = scaler.transform(test_data.reshape(-1, 1))

# Windowing
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

train_window = 12
train_inout_seq = create_inout_sequences(train_data_normalized, train_window)

# Define model
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size).to(device)

        self.linear = nn.Linear(hidden_layer_size, output_size).to(device)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size).to(device),
                            torch.zeros(1,1,self.hidden_layer_size).to(device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

# Training and optimizing model
def optimize_model(lr, hidden_dim):
    model = LSTM(input_size=1, hidden_layer_size=int(hidden_dim), output_size=1)
    model = model.to(device)
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    epochs = 100

    for i in range(epochs):
        for seq, labels in train_inout_seq:
            optimizer.zero_grad()
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size).to(device),
                            torch.zeros(1, 1, model.hidden_layer_size).to(device))

            y_pred = model(seq)

            single_loss = loss_function(y_pred, labels)
            single_loss.backward()
            optimizer.step()

    return -single_loss.item()

bounds = {'lr': (1e-4, 1e-2), 'hidden_dim': (32, 256)}
optimizer = BayesianOptimization(f=optimize_model, pbounds=bounds, random_state=42)
optimizer.maximize(init_points=10, n_iter=20)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))


|   iter    |  target   | hidden... |    lr     |
-------------------------------------------------
| 1         | -0.0337   | 115.9     | 0.009512  |
| 2         | -0.000738 | 196.0     | 0.006027  |
| 3         | -0.000600 | 66.95     | 0.001644  |
| 4         | -0.000112 | 45.01     | 0.008675  |
| 5         | -0.007345 | 166.6     | 0.00711   |
| 6         | -0.01436  | 36.61     | 0.009702  |
| 7         | -0.000843 | 218.5     | 0.002202  |
| 8         | -0.002626 | 72.73     | 0.001916  |
| 9         | -0.001365 | 100.2     | 0.005295  |
| 10        | -0.003337 | 128.8     | 0.002983  |
| 11        | -0.000146 | 50.31     | 0.004637  |
| 12        | -0.01495  | 207.1     | 0.007023  |
